In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
import warnings
warnings.filterwarnings('ignore')
import re

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity
from tensorflow import keras
from tqdm import tqdm

In [2]:

p_movies = "C:/Users/vazhitenev/PycharmProjects/Contest2023/oneproj/train/movies.csv"
p_countries = "C:/Users/vazhitenev/PycharmProjects/Contest2023/oneproj/train/countries.csv"
p_genres = "C:/Users/vazhitenev/PycharmProjects/Contest2023/oneproj/train/genres.csv"
p_staff = "C:/Users/vazhitenev/PycharmProjects/Contest2023/oneproj/train/staff.csv"
p_logs = "C:/Users/vazhitenev/PycharmProjects/Contest2023/oneproj/train/logs.csv"
'''
p_movies = "C:/Users/Vergas\Contest2023/train/movies.csv"
p_countries = "C:/Users/Vergas/Contest2023/train/countries.csv"
p_genres = "C:/Users/Vergas/Contest2023/train/genres.csv"
p_staff = "C:/Users/Vergas/Contest2023/train/staff.csv"
p_logs = "C:/Users/Vergas/Contest2023/train/logs.csv"
'''
movies = pd.read_csv(p_movies)
countries = pd.read_csv(p_countries)
genres = pd.read_csv(p_genres)
staff = pd.read_csv(p_staff)
logs = pd.read_csv(p_logs)

In [3]:
#удаляем из списка фильмо те, которые не были опубликованы
movies = movies[~movies['date_publication'].isnull()]
# переводим тип float to intager
logs['movie_id'] = logs['movie_id'].astype(int)
logs['duration'] = logs['duration'].astype(int)
logs['datetime'] = pd.to_datetime(logs['datetime'])

## Split для теста

In [4]:
# set dates params for filter
MAX_DATE = logs['datetime'].max()
MIN_DATE = logs['datetime'].min()
TEST_INTERVAL_DAYS = 14
TEST_MAX_DATE = MAX_DATE - dt.timedelta(days = TEST_INTERVAL_DAYS)
''' 
print(f"min date in filtered interactions: {MAX_DATE}")
print(f"max date in filtered interactions:: {MIN_DATE}")
print(f"test max date to split:: {TEST_MAX_DATE}")
'''

' \nprint(f"min date in filtered interactions: {MAX_DATE}")\nprint(f"max date in filtered interactions:: {MIN_DATE}")\nprint(f"test max date to split:: {TEST_MAX_DATE}")\n'

In [5]:
logs_test = logs.loc[logs['datetime'] >= TEST_MAX_DATE]
logs_test = logs_test[["user_id", "movie_id"]].drop_duplicates().rename(columns={'movie_id':'item_id'})
logs_train = logs.loc[logs['datetime'] < TEST_MAX_DATE]
logs_test = logs_test[logs_test['user_id'].isin(logs_train['user_id'].unique().tolist())]

In [6]:
test_items = logs_test.groupby('user_id')['item_id'].unique().apply(list).reset_index(name='item_list_test')
train_items= logs_train.groupby('user_id')['movie_id'].unique().apply(list).reset_index(name='item_list_train')
common_users = pd.merge(test_items, train_items, on='user_id', how='inner')
common_users['item_list_check'] = common_users.apply(lambda row: list(set(row['item_list_test']) - set(row['item_list_train'])), axis=1)
logs_test_norm = common_users.drop(['item_list_test', 'item_list_train'], axis= 1).explode('item_list_check')
logs_test_norm = logs_test_norm[~logs_test_norm['item_list_check'].isna()].rename(columns={'item_list_check':'item_id'})
logs_test_norm['interaction'] = 1

# Baseline

In [8]:
pop_movie = list(logs_train['movie_id'].value_counts().sort_values(ascending=False).index)

In [9]:
def top_u(uid, l_test, pop_movie):
    watched = list(l_test[l_test['user_id']==uid].movie_id)
    pop_movie = [x for x in pop_movie if x not in watched][:20]
    rank_df = pd.DataFrame({
        'user_id': np.repeat(uid, 20),
        'item_id': pop_movie,
        'rank': np.arange(1, 21)
    })
    return rank_df


bl_pred = pd.concat([pd.DataFrame(top_u(x, logs_train, pop_movie)) for x in logs_test['user_id'].unique()], ignore_index=True)

# Основная модель

## Подготовка данных

In [7]:
# Готовим фичи пользователей
users_df = logs_train.groupby('user_id', as_index=False).agg({'movie_id':'nunique', 'duration':'sum'}).rename(columns={'movie_id':'movie_count', 'duration':'sum_duration'})
users_df['movie_quantile']   =pd.qcut(users_df['movie_count'].rank(method='first'), q=4, precision=0)
users_df['duration_quantile']=pd.qcut(users_df['sum_duration'].rank(method='first'), q=4, precision=0)
user_cat_feats = ["movie_quantile", "duration_quantile"]

# из исходного датафрейма оставим только item_id - этот признак нам понадобится позже
# для того, чтобы маппить айтемы из датафрейма с фильмами с айтемами 
# из датафрейма с взаимодействиями
users_ohe_df = users_df.user_id
for feat in user_cat_feats:
  # получаем датафрейм с one-hot encoding для каждой категориальной фичи
  ohe_feat_df = pd.get_dummies(users_df[feat], prefix=feat)
  # конкатенируем ohe-hot датафрейм с датафреймом, 
  # который мы получили на предыдущем шаге
  users_ohe_df = pd.concat([users_ohe_df, ohe_feat_df], axis=1)

users_ohe_df.head()


,user_id,"movie_quantile_(0.0, 45743.0]","movie_quantile_(45743.0, 91485.0]","movie_quantile_(91485.0, 137227.0]","movie_quantile_(137227.0, 182969.0]","duration_quantile_(0.0, 45743.0]","duration_quantile_(45743.0, 91485.0]","duration_quantile_(91485.0, 137227.0]","duration_quantile_(137227.0, 182969.0]"
0,0,0,0,0,1,0,0,0,1
1,1,0,0,0,1,0,0,0,1
2,2,0,0,0,1,0,0,0,1
3,3,0,0,0,1,0,0,0,1
4,4,0,0,0,1,0,0,0,1


In [88]:
# Готовим фичи айтемов #1
items_df = pd.DataFrame.copy(movies).rename(columns={'id':'item_id'})

item_cat_feats = ['year', 'genres', 'countries']

items_ohe_df = items_df.item_id

for feat in item_cat_feats:
  ohe_feat_df = pd.get_dummies(items_df[feat], prefix=feat)
  items_ohe_df = pd.concat([items_ohe_df, ohe_feat_df], axis=1) 

items_ohe_df.head()

,item_id,year_1895-01-01,year_1906-01-01,year_1925-01-01,year_1934-01-01,year_1937-01-01,year_1938-01-01,year_1939-01-01,year_1940-01-01,year_1941-01-01,...,"countries_[81, 121, 102, 146]","countries_[81, 121]",countries_[81],"countries_[83, 102]","countries_[83, 242, 102]",countries_[83],countries_[84],"countries_[85, 122, 121, 102]","countries_[93, 109]",countries_[]
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [153]:
# Готовим фичи айтемов #2
def top_for_columns(column, topval, prefix):
    total_count = {}
    for row in column:
        for value in row:
            total_count[value] = total_count.get(value, 0) + 1
    pc = pd.DataFrame(total_count, index=[0]).transpose().sort_values(by=0, ascending=False)
    list_columns = pc[(pc[0]>topval)].index.values.tolist()
    list_columns = [prefix+elt for elt in list_columns]
    return list_columns

items_df = pd.DataFrame.copy(movies).rename(columns={'id':'item_id'})
items_df['staff_list'] = items_df.apply(lambda row: re.split(', ', row['staff'].strip('[]')), axis=1)
items_df['genres_list'] = items_df.apply(lambda row: re.split(', ', row['genres'].strip('[]')), axis=1)
items_df['countries_list'] = items_df.apply(lambda row: re.split(', ', row['countries'].strip('[]')), axis=1)

ohe_items_staff = pd.get_dummies(items_df['staff_list'].apply(pd.Series).stack(), prefix='staff').sum(level=0)
ohe_items_staff = ohe_items_staff[top_for_columns(items_df['staff_list'], 10, 'staff_')].drop('staff_', axis=1)
ohe_items_genres = pd.get_dummies(items_df['genres_list'].apply(pd.Series).stack(), prefix='genre').sum(level=0)
ohe_items_genres = ohe_items_genres[top_for_columns(items_df['genres_list'], 15, 'genre_')].drop('genre_', axis=1)
ohe_items_country = pd.get_dummies(items_df['countries_list'].apply(pd.Series).stack(), prefix='country').sum(level=0)
ohe_items_country = ohe_items_country[top_for_columns(items_df['countries_list'], 10, 'country_')].drop('country_', axis=1)

items_ohe_df = items_df.item_id
items_ohe_df = pd.concat([items_ohe_df, ohe_items_staff], axis=1)
items_ohe_df = pd.concat([items_ohe_df, ohe_items_genres], axis=1)
items_ohe_df = pd.concat([items_ohe_df, ohe_items_country], axis=1)

In [38]:
#Counter для расчета количества элементов
counter = Counter([item for sublist in items_df['staff_list'] for item in sublist])
df_count = pd.DataFrame.from_dict(counter, orient='index', columns=['count'])\
                        .reset_index().rename(columns={'item':'item_id'})\
                        .sort_values(by=['count'], ascending=False)
df_count

,index,count
571,,395
104,9960,38
179,3245,33
2950,10419,32
132,26989,32
...,...,...
12081,24672,1
12079,32676,1
12077,34020,1
12076,28069,1


In [191]:
'''
Собираем матрицу взаимодействий
В датасете взаимодействий есть непопулярные фильмы и малоактивные пользователи. Кроме того, в таблице взаимодействий есть события с низким качеством взаимодействия - когда юзер начал смотреть фильм, но вскоре после начала просмотра выключил. 

Отфильтруем такие события*, малоактивных юзеров и непопулярные фильмы.

_Можете не фильтровать такие события, тогда у вас будет больше негативных примеров._
'''
interactions_df = logs_train.groupby(['user_id', 'movie_id'], as_index=False).agg({'datetime':'max', 'duration':'sum'}).rename(columns={'movie_id':'item_id', 'duration':'total_dur', 'datetime':'last_watch_dt'})

print(f"N users before: {interactions_df.user_id.nunique()}")
print(f"N items before: {interactions_df.item_id.nunique()}\n")

# отфильтруем все события взаимодействий, в которых пользователь посмотрел
# фильм менее чем на 35 процентов
# замена на 10 минут
interactions_df = interactions_df[interactions_df.total_dur > 1200]

# соберем всех пользователей, которые посмотрели 
# больше 10 фильмов (можете выбрать другой порог)
valid_users = []

c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
  if entries > 5:
    valid_users.append(user_id)

# и соберем все фильмы, которые посмотрели больше 10 пользователей
valid_items = []

c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
  if entries > 50:
    valid_items.append(item_id)

# отбросим непопулярные фильмы и неактивных юзеров
interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

print(f"N users after: {interactions_df.user_id.nunique()}")
print(f"N items after: {interactions_df.item_id.nunique()}")

N users before: 182969
N items before: 5237

N users after: 38012
N items after: 2484


In [192]:
'''
После фильтрации может получиться так, что некоторые айтемы/юзеры есть в датасете взаимодействий, но при этом они отсутствуют в датасетах айтемов/юзеров или наоборот. 
Поэтому найдем id айтемов и id юзеров, которые есть во всех датасетах и оставим только их.
'''
common_users = set(interactions_df.user_id.unique()).intersection(set(users_ohe_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_ohe_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_ohe_df = items_ohe_df[items_ohe_df.item_id.isin(common_items)]
users_ohe_df = users_ohe_df[users_ohe_df.user_id.isin(common_users)]

21323
1596


Соберем взаимодействия в матрицу user*item так, чтобы в строках этой матрицы были user_id, в столбцах - item_id, а на пересечениях строк и столбцов - единица, если пользователь взаимодействовал с айтемом и ноль, если нет.

Такую матрицу удобно собирать в numpy array, однако нужно помнить, что numpy array индексируется порядковыми индексами, а нам же удобнее использовать item_id и user_id.

Создадим некие внутренние индексы для user_id и item_id - uid и iid. Для этого просто соберем все user_id и item_id и пронумеруем их по порядку.

In [193]:
interactions_df["uid"] = interactions_df["user_id"].astype("category")
interactions_df["uid"] = interactions_df["uid"].cat.codes

interactions_df["iid"] = interactions_df["item_id"].astype("category")
interactions_df["iid"] = interactions_df["iid"].cat.codes

print(sorted(interactions_df.iid.unique())[:5])
print(sorted(interactions_df.uid.unique())[:5])
interactions_df.head()

[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


,user_id,item_id,last_watch_dt,total_dur,uid,iid
1,0,74,2023-05-28 06:38:40.736898+03:00,31497,0,10
2,0,107,2023-05-31 20:35:46.239771+03:00,92347,0,14
5,0,282,2023-05-31 20:29:44.903470+03:00,16250,0,53
9,0,840,2023-05-31 20:43:47.751377+03:00,199321,0,159
13,0,927,2023-04-24 22:32:16.184274+03:00,3621,0,171


Наконец соберем и отнормируем матрицу взаимодействий:

In [194]:
interactions_vec = np.zeros((interactions_df.uid.nunique(), 
                             interactions_df.iid.nunique())) 

for user_id, item_id in zip(interactions_df.uid, interactions_df.iid):
    interactions_vec[user_id, item_id] += 1


res = interactions_vec.sum(axis=1)
for i in range(len(interactions_vec)):
    interactions_vec[i] /= res[i]


In [195]:
print(interactions_df.item_id.nunique())
print(items_ohe_df.item_id.nunique())
print(interactions_df.user_id.nunique())
print(users_ohe_df.user_id.nunique())

set(items_ohe_df.item_id.unique()) - set(interactions_df.item_id.unique())

1596
1596
21323
21323


set()

Для того, чтобы можно было удобно превратить iid/uid в item_id/user_id и наоборот соберем словари 

{iid: item_id}, {uid: user_id} и {item_id: iid}, {user_id: uid}.

In [196]:
iid_to_item_id = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("iid").to_dict()["item_id"]
item_id_to_iid = interactions_df[["iid", "item_id"]].drop_duplicates().set_index("item_id").to_dict()["iid"]

uid_to_user_id = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("uid").to_dict()["user_id"]
user_id_to_uid = interactions_df[["uid", "user_id"]].drop_duplicates().set_index("user_id").to_dict()["uid"]

И проиндексируем датасеты users_ohe_df и items_ohe_df по внутренним айди:

In [197]:
items_ohe_df["iid"] = items_ohe_df["item_id"].apply(lambda x: item_id_to_iid[x])
items_ohe_df = items_ohe_df.set_index("iid")

users_ohe_df["uid"] = users_ohe_df["user_id"].apply(lambda x: user_id_to_uid.get(x, 'unknown'))
users_ohe_df = users_ohe_df.set_index("uid")

## DSSM starter's pack

Возьмем вектор юзера (anchor) и векторы двух айтемов - "хорошего" и "плохого" (positive и negative). Хороший айтем - это тот, который пользователь уже посмотрел, а в качестве плохого возьмем любой случайный айтем из датасета. Затем посчитаем расстояния:
1. между вектором юзера и вектором "хорошего" айтема
2. между вектором юзера и вектором "плохого" айтема

_Значением функции потерь будет разность между первым и вторым расстоянием._

In [198]:
def triplet_loss(y_true, y_pred, n_dims=128, alpha=0.4):
    # будем ожидать, что на вход функции прилетит три сконкатенированных 
    # вектора - вектор юзера и два вектора айтема
    anchor = y_pred[:, 0:n_dims]
    positive = y_pred[:, n_dims:n_dims*2]
    negative = y_pred[:, n_dims*2:n_dims*3]

    # считаем расстояния от вектора юзера до вектора хорошего айтема
    pos_dist = K.sum(K.square(anchor - positive), axis=1)
    # и до плохого
    neg_dist = K.sum(K.square(anchor - negative), axis=1)

    # считаем лосс
    basic_loss = pos_dist - neg_dist + alpha
    loss = K.maximum(basic_loss, 0.0) # возвращаем ноль, если лосс отрицательный
 
    return loss

In [199]:
def generator(items, users, interactions, batch_size=1024):
    while True:
        uid_meta = []
        uid_interaction = []
        pos = []
        neg = []
        for _ in range(batch_size):
            # берем рандомный uid
            uid_i = randint(0, interactions.shape[0]-1)
            # id хорошего айтема
            pos_i = np.random.choice(range(interactions.shape[1]), p=interactions[uid_i])
            # id плохого айтема
            neg_i = np.random.choice(range(interactions.shape[1]))
            # фичи юзера
            uid_meta.append(users.iloc[uid_i])
            # вектор айтемов, с которыми юзер взаимодействовал
            uid_interaction.append(interactions_vec[uid_i])
            # фичи хорошего айтема
            pos.append(items.iloc[pos_i])
            # фичи плохого айтема
            neg.append(items.iloc[neg_i])
            
        yield [np.array(uid_meta), np.array(uid_interaction), np.array(pos), np.array(neg)], [np.array(uid_meta), np.array(uid_interaction)]


Посмотрим, что получается:

In [200]:
# инициализируем генератор
gen = generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                users=users_ohe_df.drop(["user_id"], axis=1), 
                interactions=interactions_vec)

ret = next(gen)


print(f"вектор фичей юзера: {ret[0][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[0][1].shape}")
print(f"вектор 'хорошего' айтема: {ret[0][2].shape}")
print(f"вектор 'плохого' айтема: {ret[0][3].shape}")
print()
print(f"вектор фичей юзера: {ret[1][0].shape}")
print(f"вектор взаимодействий юзера с айтемами: {ret[1][1].shape}")

вектор фичей юзера: (1024, 8)
вектор взаимодействий юзера с айтемами: (1024, 1596)
вектор 'хорошего' айтема: (1024, 442)
вектор 'плохого' айтема: (1024, 442)

вектор фичей юзера: (1024, 8)
вектор взаимодействий юзера с айтемами: (1024, 1596)


## Собираем модель

Для того, чтобы обучить модель используя триплет лосс, нам нужно получить три вектора - вектор юзера, вектор "хорошего" айтема и вектор "плохого" айтема. Для этого нам нужно две модели ("хороший" и "плохой" айтем будут семплироваться одной и той же моделью).  

Модель юзера будет иметь два входа:
- вход для фичей юзера (фичи из users_ohe_df)
- вход для вектора айтемов, которые посмотрел юзер (строка interactions_vec, которая соответствует uid конкретного юзера)

Выход модели юзера будет размерностью N_FACTORS.

У модели айтема будет один вход для фичей айтема (из items_ohe_df) и один выход также размерностью N_FACTORS.

Общая архитектура будет вот такой: 
- есть модель юзера и модель айтема
- обе модели семплируют юзер и айтем-фичи во внутреннее пространство размерностью N_FACTORS
- модель айтема семплирует два айтема - "хороший" и "плохой"
- в итоге получается три вектора размерностью N_FACTORS (вектор юзера, вектор "хорошего" айтема и вектор "плохого" айтема)
- затем полученные векторы конкатенируются, по ним считается triplet loss
- profit

Для того, чтобы собрать модель, помимо размерности внутреннего пространства, нам нужно знать еще размерность вектора юзера и вектора айтема. Зададим их сразу.

In [201]:
N_FACTORS = 128

# в датасетах есть столбец user_id/item_id, помним, что он не является фичей для обучения!
ITEM_MODEL_SHAPE = (items_ohe_df.drop(["item_id"], axis=1).shape[1], ) 
USER_META_MODEL_SHAPE = (users_ohe_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (interactions_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (442,)
USER_META_MODEL_SHAPE: (8,)
USER_INTERACTION_MODEL_SHAPE: (1596,)


In [202]:
def item_model(n_factors=N_FACTORS):
    # входной слой
    inp = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
    
    # полносвязный слой
    layer_1 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                               kernel_regularizer=keras.regularizers.l2(1e-6),
                               activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp)

    # делаем residual connection - складываем два слоя, 
    # чтобы градиенты не затухали во время обучения
    layer_2 = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1)
    
    add = keras.layers.Add()([layer_1, layer_2])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(add)
    
    return keras.models.Model(inp, out)


def user_model(n_factors=N_FACTORS):
    # входной слой для вектора фичей юзера (из users_ohe_df)
    inp_meta = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
    # входной слой для вектора просмотров (из iteractions_vec)
    inp_interaction = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

    # полносвязный слой
    layer_1_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_meta)

    layer_1_interaction = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(inp_interaction)

    # делаем residual connection - складываем два слоя,
    # чтобы градиенты не затухали во время обучения
    layer_2_meta = keras.layers.Dense(N_FACTORS, activation='elu', use_bias=False,
                                 kernel_regularizer=keras.regularizers.l2(1e-6),
                                 activity_regularizer=keras.regularizers.l2(l2=1e-6))(layer_1_meta)
    

    add = keras.layers.Add()([layer_1_meta, layer_2_meta])
    
    # конкатенируем вектор фичей с вектором просмотров
    concat_meta_interaction = keras.layers.Concatenate()([add, layer_1_interaction])
    
    # выходной слой
    out = keras.layers.Dense(N_FACTORS, activation='linear', use_bias=False,
                             kernel_regularizer=keras.regularizers.l2(1e-6),
                             activity_regularizer=keras.regularizers.l2(l2=1e-6))(concat_meta_interaction)
    
    return keras.models.Model([inp_meta, inp_interaction], out)

# инициализируем модели юзера и айтема
i2v = item_model()
u2v = user_model()

# вход для вектора фичей юзера (из users_ohe_df)
ancor_meta_in = keras.layers.Input(shape=USER_META_MODEL_SHAPE)
# вход для вектора просмотра юзера (из interactions_vec)
ancor_interaction_in = keras.layers.Input(shape=USER_INTERACTION_MODEL_SHAPE)

# вход для вектора "хорошего" айтема
pos_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)
# вход для вектора "плохого" айтема
neg_in = keras.layers.Input(shape=ITEM_MODEL_SHAPE)

# получаем вектор юзера
ancor = u2v([ancor_meta_in, ancor_interaction_in])
# получаем вектор "хорошего" айтема
pos = i2v(pos_in)
# получаем вектор "плохого" айтема
neg = i2v(neg_in)

# конкатенируем полученные векторы
res = keras.layers.Concatenate(name="concat_ancor_pos_neg")([ancor, pos, neg])

# собираем модель
model = keras.models.Model([ancor_meta_in, ancor_interaction_in, pos_in, neg_in], res)

In [203]:
model_name = 'recsys_resnet_linear'

# логируем процесс обучения в тензорборд
t_board = keras.callbacks.TensorBoard(log_dir=f'runs/{model_name}')

# уменьшаем learning_rate, если лосс долго не уменьшается (в течение двух эпох)
decay = keras.callbacks.ReduceLROnPlateau(monitor='loss', patience=2, factor=0.8, verbose=1)

# сохраняем модель после каждой эпохи, если лосс уменьшился
check = keras.callbacks.ModelCheckpoint(filepath=model_name + '/epoch{epoch}-{loss:.2f}.h5', monitor="loss")


In [204]:
# компилируем модель, используем оптимайзер Adam и triplet loss
opt = keras.optimizers.Adam(lr=0.001)
model.compile(loss=triplet_loss, optimizer=opt)

Посмотрим, что получилось:

In [168]:
# модель айтема
item_model().summary()

Model: "model_33"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_68 (InputLayer)       [(None, 442)]                0         []                            
                                                                                                  
 dense_91 (Dense)            (None, 128)                  56576     ['input_68[0][0]']            
                                                                                                  
 dense_92 (Dense)            (None, 128)                  16384     ['dense_91[0][0]']            
                                                                                                  
 add_26 (Add)                (None, 128)                  0         ['dense_91[0][0]',            
                                                                     'dense_92[0][0]']     

In [169]:
# модель юзера
user_model().summary()

Model: "model_34"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_69 (InputLayer)       [(None, 8)]                  0         []                            
                                                                                                  
 dense_94 (Dense)            (None, 128)                  1024      ['input_69[0][0]']            
                                                                                                  
 dense_96 (Dense)            (None, 128)                  16384     ['dense_94[0][0]']            
                                                                                                  
 input_70 (InputLayer)       [(None, 2894)]               0         []                            
                                                                                           

In [170]:
# общая модель
model.summary()

Model: "model_32"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_64 (InputLayer)       [(None, 8)]                  0         []                            
                                                                                                  
 input_65 (InputLayer)       [(None, 2894)]               0         []                            
                                                                                                  
 input_66 (InputLayer)       [(None, 442)]                0         []                            
                                                                                                  
 input_67 (InputLayer)       [(None, 442)]                0         []                            
                                                                                           

In [205]:
# начинаем обучение, не забывая дропнуть столбцы item_id и user_id 
# из датафреймов при инициализации генератора.

# batch_size можно (и лучше) поставить побольше, если вы не органичены в ресурсах

model.fit(generator(items=items_ohe_df.drop(["item_id"], axis=1), 
                    users=users_ohe_df.drop(["user_id"], axis=1), 
                    interactions=interactions_vec,
                    batch_size=64), 
          steps_per_epoch=100, 
          epochs=30, 
          initial_epoch=0,
          callbacks=[decay, t_board, check]
)

Epoch 1/30
100/100 [==============================] - 4s 30ms/step - loss: 0.5822 - lr: 0.0010
Epoch 2/30
100/100 [==============================] - 3s 30ms/step - loss: 0.3219 - lr: 0.0010
Epoch 3/30
100/100 [==============================] - 3s 29ms/step - loss: 0.2604 - lr: 0.0010
Epoch 4/30
100/100 [==============================] - 3s 31ms/step - loss: 0.2280 - lr: 0.0010
Epoch 5/30
100/100 [==============================] - 3s 31ms/step - loss: 0.2148 - lr: 0.0010
Epoch 6/30
100/100 [==============================] - 4s 39ms/step - loss: 0.2120 - lr: 0.0010
Epoch 7/30
100/100 [==============================] - 4s 36ms/step - loss: 0.1977 - lr: 0.0010
Epoch 8/30
100/100 [==============================] - 3s 35ms/step - loss: 0.1911 - lr: 0.0010
Epoch 9/30
100/100 [==============================] - 3s 31ms/step - loss: 0.1952 - lr: 0.0010
Epoch 10/30
100/100 [==============================] - 4s 37ms/step - loss: 0.1842 - lr: 0.0010
Epoch 11/30
100/100 [============================

## Inference (пример на слуяайном)

Отлично! Мы подготовили данные, собрали модель по архитектуре DSSM и обучили ее.
Теперь возьмем случайного юзера и случайный айтем. Как понять, насколько этот айтем релевантен юзеру?

Нужно:
- получить вектор айтема;
- получить вектор юзера;
- посчитать расстояние между ними.

Это расстояние и есть мера релевантности.


In [58]:
# берем рандомного юзера
rand_uid = np.random.choice(list(users_ohe_df.index))

# получаем фичи юзера и вектор его просмотров айтемов
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).iloc[rand_uid.astype(int)]
user_interaction_vec = interactions_vec[rand_uid.astype(int)]

# берем рандомный айтем
rand_iid = np.random.choice(list(items_ohe_df.index))
# получаем фичи айтема
item_feats = items_ohe_df.drop(["item_id"], axis=1).iloc[rand_iid]

# получаем вектор юзера
user_vec = u2v.predict([np.array(user_meta_feats).reshape(1, -1), 
                        np.array(user_interaction_vec).reshape(1, -1)])

# и вектор айтема
item_vec = i2v.predict(np.array(item_feats).reshape(1, -1))

# считаем расстояние между вектором юзера и вектором айтема
from sklearn.metrics.pairwise import euclidean_distances as ED

ED(user_vec, item_vec)

1/1 [==============================] - 0s 64ms/step


array([[1.3160759]], dtype=float32)

In [59]:
# получаем фичи всех айтемов
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
# получаем векторы всех айтемов
items_vecs = i2v.predict(items_feats)

# считаем расстояния
dists = ED(user_vec, items_vecs)

94/94 [==============================] - 0s 1ms/step


In [60]:
top5_iids = np.argsort(dists, axis=1)[0][:20]
top5_iids

array([ 267,  180,  808, 2088, 1263,  242, 2430,  254, 1823, 1997,  801,
       1050,  799,  175, 1659, 2701,  956, 2213, 1368, 2354], dtype=int64)

Осталось конвертировать внутренние iid в ~~id здорового человека~~ item_id:

In [61]:
top5_item_ids = [iid_to_item_id[iid] for iid in top5_iids]
df_pred_u = pd.DataFrame({'item_id': top5_item_ids})
df_pred_u['rank'] = df_pred_u.index+1
df_pred_u['user_id'] = rand_uid

In [62]:
recommended_titles = items_df.loc[items_df.item_id.isin(top5_item_ids)].name
recommended_titles

475                                          2012
484                             Аватар: Путь воды
658                                         Мумия
685                                  Чёрная Вдова
728     Доктор Стрэндж: В мультивселенной безумия
2052                                       Дэдпул
2057                                  Чёрный Адам
2074                                      Морбиус
2460                               Крепкий орешек
2679                                       Вечныe
3213                           Мумия возвращается
3493              Отряд самоубийц: Миссия навылет
4178                       Бегущий по лезвию 2049
4603                   Kingsman: Секретная служба
5042                       Хроники хищных городов
5281                Шан-Чи и легенда десяти колец
5584                            Миссия «Серенити»
5918                           Тор: Любовь и гром
6099           G. I. Joe. Бросок кобры: Снейк Айз
6758                    Люди Икс: Последняя битва


## Inference для всей выборки

In [206]:
from sklearn.metrics.pairwise import euclidean_distances as ED

# Получаем фичи всех айтемов и векторы всех айтемов заранее
items_feats = items_ohe_df.drop(["item_id"], axis=1).to_numpy()
items_vecs = i2v.predict(items_feats)

# Получаем фичи юзеров и векторы их просмотров айтемов
#user_meta_feats = users_ohe_df.drop('unknown', axis=0).drop(["user_id"], axis=1).to_numpy()
user_meta_feats = users_ohe_df.drop(["user_id"], axis=1).to_numpy()

user_interaction_vecs = interactions_vec  # Нет необходимости использовать to_numpy()

# Получаем векторы юзеров
user_vecs = u2v.predict([user_meta_feats, user_interaction_vecs])

# Считаем расстояния между векторами юзеров и векторами всех айтемов
dists = ED(user_vecs, items_vecs)

# Находим ТОП-20 айтемов для каждого пользователя
top20_iids = np.argsort(dists, axis=1)[:, :20]
top20_item_ids = np.vectorize(iid_to_item_id.get)(top20_iids)  # Применяем маппинг iid в item_id

# Создаем DataFrame с результатами
df_pred = pd.DataFrame({
    'user_id': np.repeat(users_ohe_df.index, 20)[:len(top20_item_ids.flatten())],
    'item_id': top20_item_ids.flatten(),
    'rank': np.tile(np.arange(1, 21), len(users_ohe_df))[:len(top20_item_ids.flatten())]
})

667/667 [==============================] - 1s 2ms/step


## Расчет метрики МАР

In [207]:
# baseline == 0.0218218110206651
# map_v2(logs_test_norm, bl_pred, 20)
# фичи айтемов #1 == 0.00586553605484825
# фичи айтемов #1 == 0.005702631349202798; >1800sec; >10 films; >20 users
# фичи айтемов #2 == 0.004860998200328909; >600sec;  >2 films;  >10 users; topval staff=10, genres=15, counties=10
# фичи айтемов #2 == 0.005121920502313525; >1800sec; >10 films; >20 users; topval staff=20, genres=20, counties=20
# фичи айтемов #2 == 0.0051762269717752175;>1800sec; >10 films; >20 users; topval staff=5, genres=10, counties=5
# фичи айтемов #2 == 0.005103638740674767; >1800sec; >10 films; >100 users; topval staff=10, genres=15, counties=10
# фичи айтемов #2 == 0.00616865866360225;  >1800sec; >10 films; >20 users; topval staff=10, genres=15, counties=10
# фичи айтемов #2 == 0.006182297656765647; >1200sec; >5 films;  >50  users; topval staff=10, genres=15, counties=10
map_v2(logs_test_norm, df_pred, 20)

0.006182297656765647

In [64]:
def mean_average_precision(df_true, df_pred, k=20, target_col='rank'):

    # Объединение данных по 'user_id' и 'item_id'

    df = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))

    # Отбор пользователей, которые есть и в обучающей, и в тестовой выборке
    common_users = set(df_true['user_id']).intersection(set(df_pred['user_id']))
    df = df[df.index.get_level_values('user_id').isin(common_users)]

    # Сортировка данных по 'user_id' и предсказаниям
    df = df.sort_values(by=['user_id', target_col], ascending=[True, False])

    # Инициализация переменных для расчета MAP
    total_precision = 0.0
    users_count = 0

    # Расчет MAP для каждого пользователя
    for user_id, user_df in df.groupby(level='user_id'):
        user_precision = 0.0
        relevant_items = 0

        for i, (_, row) in enumerate(user_df.head(k).iterrows(), 1):
            # Рассчитываем W = P / (N * R)
            precision = row['interaction'] / i
            user_precision += precision
            relevant_items += row['interaction']

        if relevant_items > 0:
            user_precision /= relevant_items  # Усреднение для пользователя
            total_precision += user_precision
            users_count += 1


    # Расчет среднего значения MAP по всем пользователям
    if users_count > 0:
        mean_ap = total_precision / users_count
        return mean_ap
    else:
        return 0.0


In [65]:
def apk (actual, predicted, k=10):
    if not actual:
        return 0.0
    if len(predicted)>k:
        predicted = predicted[:k]
    
    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def map_v2(df_true, df_pred, k=10):
    test_items = df_true.groupby('user_id')['item_id'].apply(list).reset_index(name='item_list_test')
    pred_items = df_pred.sort_values(by='rank').groupby('user_id')['item_id'].apply(list).reset_index(name='item_list_pred')
    merget_array = pd.merge(test_items, pred_items, on='user_id', how='inner')
    return mapk(merget_array['item_list_test'], merget_array['item_list_pred'], 20)

## скрыть расчет метрик

In [70]:
mean_average_precision(logs_test_norm, df_pred)

0.5990594263288112

In [71]:
#baseline
mean_average_precision(logs_test_norm, bl_pred)

0.6796000244990886

In [67]:
# baseline == 0.0218218110206651
map_v2(logs_test_norm, bl_pred, 20)

0.0218218110206651